<a href="https://colab.research.google.com/github/thinkhow/thinkhow.github.io/blob/master/Loading_and_process_data_SIG_Sentiment_analysis_using_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#for saving files and trained models to save RAM
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Install packages




In [ ]:
pip install praw

     |████████████████████████████████| 163kB 7.4MB/s 
     |████████████████████████████████| 71kB 7.1MB/s 


Import packages


In [ ]:
import praw
import pandas as pd

Scrape Reddit data 




In [ ]:
# Acessing the reddit api

reddit = praw.Reddit(client_id="8SMVmmj4NUo4hw",      # your client id
                     client_secret="jbmzvrX3heEFjtcDNBtIXeqUTlIT_g",  #your client secret
                     user_agent="Khoa Nguyen", #user agent name
                     username = "hopernguyen",     # your reddit username
                     password = "")     # your reddit password

In [ ]:
sub = ['Askreddit']  # make a list of subreddits you want to scrape the data from

for s in sub:
    
    subreddit = reddit.subreddit(s)   # Chosing the subreddit

In [ ]:
########################################
#   CREATING DICTIONARY TO STORE THE DATA WHICH WILL BE CONVERTED TO A DATAFRAME
########################################

#   NOTE: ALL THE POST DATA AND COMMENT DATA WILL BE SAVED IN TWO DIFFERENT
#   DATASETS AND LATER CAN BE MAPPED USING IDS OF POSTS/COMMENTS AS WE WILL 
#   BE CAPTURING ALL IDS THAT COME IN OUR WAY

# SCRAPING CAN BE DONE VIA VARIOUS STRATEGIES {HOT,TOP,etc} we will go with keyword strategy i.e using search a keyword
query = ['Gaming']
for item in query:
    post_dict = {
        "title" : [],   #title of the post
        "score" : [],   # score of the post
        "id" : [],      # unique id of the post
        "url" : [],     #url of the post
        "comms_num": [],   #the number of comments on the post
        "created" : [],  #timestamp of the post
        "body" : []         # the descriptionof post
    }
    comments_dict = {
        "comment_id" : [],      #unique comm id
        "comment_parent_id" : [],   # comment parent id
        "comment_body" : [],   # text in comment
        "comment_link_id" : []  #link to the comment
    }
    for submission in subreddit.search(query,sort = "top",limit = 1):
        post_dict["title"].append(submission.title)
        post_dict["score"].append(submission.score)
        post_dict["id"].append(submission.id)
        post_dict["url"].append(submission.url)
        post_dict["comms_num"].append(submission.num_comments)
        post_dict["created"].append(submission.created)
        post_dict["body"].append(submission.selftext)
        
        ##### Acessing comments on the post
        submission.comments.replace_more(limit = 1)
        for comment in submission.comments.list():
            comments_dict["comment_id"].append(comment.id)
            comments_dict["comment_parent_id"].append(comment.parent_id)
            comments_dict["comment_body"].append(comment.body)
            comments_dict["comment_link_id"].append(comment.link_id)
    
    post_comments = pd.DataFrame(comments_dict)

    post_comments.to_csv("./drive/MyDrive/Erdo_Project/Data/"+s+"_comments_"+ item +"subreddit.csv")
    post_data = pd.DataFrame(post_dict)
    post_data.to_csv("./drive/MyDrive/Erdos_Project/Data"+s+"_"+ item +"subreddit.csv")

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



FileNotFoundError: ignored

Scrape twitter data

SPY data (Upload kaggle file)



FinViz stock screener - Original article: https://towardsdatascience.com/stock-news-sentiment-analysis-with-python-193d4b4378d4 

This shows how to import the market data by ticker using FinViz and some easy sentiment analysis



In [ ]:
pip install twython

The variable n represents the number of articles that will be displayed for each ticker in the ‘tickers’ list. The rest of the code will not have to be manually updated and these are the only parameters you will have to change each time you run the code.

In [ ]:
# Import libraries
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from urllib.request import urlopen
from urllib.request import Request
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
# Parameters 
n = 3 #the # of article headlines displayed per ticker
tickers = ['AAPL', 'TSLA', 'AMZN']

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


we will get the news data from the FinViz website using the modules BeautifulSoup and requests. The code parses the URL for the HTML table of news and iterates through the list of tickers to gather the recent headlines for each ticker. For each inputted stock, an ‘n’ number of recent headlines is printed out so the data is easy to view.

The news are typically from yahoo, barrons, motley fool, insider monkey,...


In [ ]:
# Get Data
finviz_url = 'https://finviz.com/quote.ashx?t='
news_tables = {}

for ticker in tickers:
    url = finviz_url + ticker
    req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
    resp = urlopen(req)    
    html = BeautifulSoup(resp, features="lxml")
    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table

try:
    for ticker in tickers:
        df = news_tables[ticker]
        df_tr = df.findAll('tr')
    
        print ('\n')
        print ('Recent News Headlines for {}: '.format(ticker))
        
        for i, table_row in enumerate(df_tr):
            a_text = table_row.a.text
            td_text = table_row.td.text
            td_text = td_text.strip()
            print(a_text,'(',td_text,')')
            if i == n-1:
                break
except KeyError:
    pass



Recent News Headlines for AAPL: 
WFH opens up companies to more cyber threats: Microsoft ( May-12-21 11:04AM )
Will Bill Gates Divorce Affect These Stocks? ( 10:42AM )
4 Dow Jones Stocks To Buy And Watch In May 2021: Apple, Microsoft Sell Off ( 10:31AM )


Recent News Headlines for TSLA: 
Dow Jones Slides, Tech Stocks Dive As Yields Jump On Accelerating Inflation; Apple, Tesla Sell Off ( May-12-21 11:02AM )
ARK Invest Stocks To Buy And Watch: 6 Stocks That Cathie Wood's ARK ETFs Own; Sea Limited, Square, Tesla Slide ( 10:52AM )
UPDATE 1-Tesla says it supports standardisation of China auto industry ( 09:17AM )


Recent News Headlines for AMZN: 
WFH opens up companies to more cyber threats: Microsoft ( May-12-21 11:04AM )
Will Bill Gates Divorce Affect These Stocks? ( 10:42AM )
Amazon wins court fight over $300M EU tax order ( 10:00AM )


In [ ]:
# Iterate through the news
parsed_news = []
for file_name, news_table in news_tables.items():
    for x in news_table.findAll('tr'):
        text = x.a.get_text() 
        date_scrape = x.td.text.split()

        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        else:
            date = date_scrape[0]
            time = date_scrape[1]

        ticker = file_name.split('_')[0]
        
        parsed_news.append([ticker, date, time, text])

Each headline is analyzed for its polarity score on a scale of -1 to 1, with -1 being highly negative and highly 1 being positive.


In [ ]:

# Sentiment Analysis
analyzer = SentimentIntensityAnalyzer()

columns = ['Ticker', 'Date', 'Time', 'Headline']
news = pd.DataFrame(parsed_news, columns=columns)
scores = news['Headline'].apply(analyzer.polarity_scores).tolist()

df_scores = pd.DataFrame(scores)
news = news.join(df_scores, rsuffix='_right')

 For each ticker in the inputted list, a new DataFrame will be created that includes its headlines and their respective scores

 DataFrame will be created that includes each ticker’s mean sentiment value over all the recent news parsed.
 

In [ ]:
# View Data 
news['Date'] = pd.to_datetime(news.Date).dt.date

unique_ticker = news['Ticker'].unique().tolist()
news_dict = {name: news.loc[news['Ticker'] == name] for name in unique_ticker}

values = []
for ticker in tickers: 
    dataframe = news_dict[ticker]
    dataframe = dataframe.set_index('Ticker')
    dataframe = dataframe.drop(columns = ['Headline'])
    print ('\n')
    print (dataframe.head())
    
    mean = round(dataframe['compound'].mean(), 2)
    values.append(mean)
    
df = pd.DataFrame(list(zip(tickers, values)), columns =['Ticker', 'Mean Sentiment']) 
df = df.set_index('Ticker')
df = df.sort_values('Mean Sentiment', ascending=False)
print ('\n')
print (df)

NameError: ignored

Recurrent Neural Networks and Long Short Term Memory (RNN and LSTM). The idea is we use the sentiment analysis to produce the score (unsupervised), use RNN to fit the score to the market movement (supervised). For testing, we first calculate the sentiment for the test info (unsupervised), feed it to the trained RRN, and predict the market. 
Reference: https://arxiv.org/pdf/1705.02447.pdf
Github: https://github.com/irfanICMLL/EMM-for-stock-prediction



Neural Network

In [ ]:
from torch import nn
from torch import optim

In [ ]:
#once training and test data is defined, using this cell to load into the dataloader for automatic batching
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [ ]:
n=100 #number of posts/news examined by sentiment analysis for a stock at particular time
input_size = n+1 #change based on size of input vector for X (i.e. sentiment analysis for n posts, plus 1 for volatility )

#neural network inside sequential container (in case we want to add more layers)
model = nn.Sequential(
    nn.LSTM(input_size=input_size, hidden_size=25), #hidden size is the output dimension of the LSTM cell
    nn.Linear(25,1) #(input, output) want a seperate output layer so that we can have more lstm cells (i.e. neurons) in the hidden layer
)

In [ ]:
learning_rate =0.001 #default for adam
optimizer = optim.Adam(model.parameters(), lr=learning_rate) #using default first and second moment values (i.e. betas in documentation)

In [ ]:
loss_fn = nn.MSELoss() #Mean squared error loss since output is continuous 

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X,y) in enumerate(dataloader):
    pred =model(X)  #we might need to add something for the recurrent hidden output idk, this is just one of the examples
    loss =loss_fn(pred, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
#https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html
def test_loop(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  test_loss, correct = 0, 0

  with torch.no_grad():
      for X, y in dataloader:
        pred = model(X)
        test_loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= size
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
#https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html
epochs=10
for i in range(epochs):
  print(f"Epoch {i}\n---------------------")
  train_loop(train_dataloader, model, loss_fn, optimizer)
  test_loop(test_dataloader, model, loss_fn) #validation set function call to check generalization